In [23]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
import pandas as pd
import math
from datetime import datetime, timedelta

driver = webdriver.Chrome(executable_path='./chromedriver')

driver.get("https://www.indeed.com/advanced_search")

driver.implicitly_wait(3)

# search analyst
search_job = driver.find_element_by_xpath('//input[@id="as_and"]')
search_job.send_keys(['Analyst'])

# search location
searchLocation = driver.find_element_by_xpath('//input[@id="where"]')
searchLocation.clear()
searchLocation.send_keys("Columbus,OH")

# set display limit of 50 results per page
display_limit = driver.find_element_by_xpath(
    '//select[@id="limit"]//option[@value="50"]')
display_limit.click()
driver.implicitly_wait(3)

# sort display with date
display_sort = driver.find_element_by_xpath(
    '//select[@id="sort"]//option[@value="date"]')
display_sort.click()
driver.implicitly_wait(3)

# limited to 15 days
result_age = driver.find_element_by_xpath(
    '//select[@id="fromage"]//option[@value="7"]')
result_age.click()
driver.implicitly_wait(3)

# push search button
search_button = driver.find_element_by_xpath('//*[@id="fj"]')
search_button.click()
driver.implicitly_wait(3)

# Get exact search result amount
search_count = driver.find_element_by_xpath('//div[@id="searchCount"]').text

def sub_str(s, start, end):
    return s[s.find(start)+len(start):s.rfind(end)]
pages = math.ceil(int(sub_str(search_count, "of ", " jobs"))/50)
print("total pages: ", pages)

total pages:  3


In [48]:
%%time

titles = []
companies = []
links = []
salaries = []
locations = []
descriptions = []
dates = []

for i in range(0, pages):

    job_card = driver.find_elements_by_xpath(
        '//div[contains(@class,"clickcard")]')
    
    print(len(job_card))

    for job in job_card:

        # Title
        try:
            title = job.find_element_by_xpath('.//h2[@class="title"]//a').text
        except:
            title = job.find_element_by_xpath(
                './/h2[@class="title"]//a').get_attribute(name="title")
        titles.append(title)

#         # Link
#         links.append(job.find_element_by_xpath(
#             './/h2[@class="title"]//a').get_attribute(name="href"))
        
#         # company
#         companies.append(job.find_element_by_xpath(
#             './/span[@class="company"]').text)

        # salary
#         try:
#             salary = job.find_element_by_xpath(
#                 './/span[@class="salaryText"]').text
#         except:
#             salary = "None"
#         salaries.append(salary)

#         # location
#         try:
#             location = job.find_element_by_xpath(
#                 './/span[contains(@class,"location")]').text
#         except:
#             location = "None"
#         locations.append(location)

#         # date posted
#         try:
#             date = job.find_element_by_xpath('.//span[@class="date "]').text
#             word_list = date.split()

#             if word_list[0].isdigit():
#                 num = int(word_list[0])
#                 new_date_prior = datetime.now() - timedelta(days=num)
#                 date = new_date_prior.strftime("%m/%d/%Y")
#             elif word_list[0] == '30+':
#                 new_date_prior = datetime.now()-timedelta(days=30)
#                 date = new_date_prior.strftime("%m/%d/%Y")+" prior"
#             else:
#                 date = datetime.now().strftime("%m/%d/%Y")
#         except:
#             date = "None"

#         dates.append(date)

    print("Finish the page: {}!".format(str(i)))
    
    # Flip to next page
    try:
        next_page = driver.find_element_by_xpath(
            '//a[@aria-label={}]//span[@class="pn"]'.format(i+2))
        next_page.click()
    except:
        next_page = "None"

50
Finish apage: 0!
50
Finish apage: 1!
50
Finish apage: 2!
CPU times: user 167 ms, sys: 13.7 ms, total: 181 ms
Wall time: 6.72 s


In [47]:
titles[100:]

['Sr Financial Analyst, Digital Marketing',
 'Business Intelligence Business Analyst - Senior',
 'Senior Business Analyst',
 'LIS ANALYST NEEDED – PREFER ATLAS OR SUNQUEST',
 'IAM Analyst',
 'Project Controls Analyst Assc',
 'Accounting Analyst (Utility Analyst 2)',
 'Billing Analyst',
 'Telecom Billing Financial Analyst (TEMS)',
 'System Analyst - Instructional Designer',
 'Business Data Analyst',
 'Digital Analyst – Google Ads',
 'Sr. Prospect Research Officer & Data Analyst',
 'Data Analyst',
 'Supply Chain Analyst',
 'Product Analyst, Associate',
 'Senior IT Business Systems Analyst',
 'Sr Analyst, Business Analysis',
 'Sr. Financial Analyst, Financial Reporting',
 'IT Business Analyst',
 'Senior Analyst, Inside Sales Account Management',
 'Senior Analyst, Supply Chain',
 'Professional Billing System Analyst',
 'Change, Analytics, and CAO - Change the Bank Sr. Business An...',
 'US Cyber Security Technology Engineering Analyst',
 'Func System Analyst Assc - Analyst (Account Premise

In [33]:
titles[140:]

['Business Analyst & Reporting Analyst',
 'Data Analyst II',
 'Root Cause Analyst - Full Time Work From Home',
 'Requirement Analyst',
 'Performance Data Analyst - Various locations',
 'Requirement/Business Analyst',
 'RA CMC Analyst II',
 'Strategic Business Analyst II--CCB Corrections Data Validati...',
 'Program Analyst',
 'GIS Analyst']

In [15]:
# Despcriptions
for link in links:
    try:
        driver.get(link)
        jd = driver.find_element_by_xpath(
            '//div[@id="jobDescriptionText"]').text
    except:
        jd = "None"

    descriptions.append(jd)


df_da = pd.DataFrame()
df_da['Title'] = titles
df_da['Company'] = companies
df_da['Location'] = locations
df_da['Link'] = links
df_da['Salary'] = salaries
df_da['Date'] = dates
df_da['Description'] = descriptions

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=54263): Max retries exceeded with url: /session/989d8995c60775d9797303de55588e72/element/38d04ef5-04db-4435-b24d-fbefe603018e/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fcbda6a3970>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [1]:
# save to feather file
filename = datetime.now().strftime("%m_%d_%Y")
df_da.to_feather("./source/"+filename)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=87.0.4280.88)
